<a href="https://colab.research.google.com/github/Edwin372/Thesis/blob/main/Vudenc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/LauraWartschinski/VulnerabilityDetection.git
!pip install torchmetrics
!mv /content/VulnerabilityDetection/Code/w2v /content
!mv /content/VulnerabilityDetection/Code/myutils.py /content
!mv /content/VulnerabilityDetection/Code/data /content
!unrar e /content/w2v/othermodels.rar /content/w2v
!unrar e /content/w2v/pythontraining_withString.rar /content/w2v
!pip install wandb

Cloning into 'VulnerabilityDetection'...
remote: Enumerating objects: 1250, done.
remote: Counting objects: 100% (158/158), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 1250 (delta 44), reused 133 (delta 25), pack-reused 1092
Receiving objects: 100% (1250/1250), 294.55 MiB | 17.52 MiB/s, done.
Resolving deltas: 100% (473/473), done.
Checking out files: 100% (206/206), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 31.3 MB/s 

UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/w2v/othermodels.rar

Extracting  /content/w2v/word2vec_withoutString10-1-200.model              0%  1%  2%  3%  OK 
Extracting  /content/w2v/word2vec_withoutString10-5-200.model              3%  4%  5%  6%  OK 
Extracting  /content/w2v/word2vec_withoutString10-10-200.model             6%  7%

In [2]:
import torch
import torch.nn as nn
import myutils
import sys
import os
from sklearn.metrics import accuracy_score 
from sklearn.metrics import precision_score 
from sklearn.metrics import recall_score 
from sklearn.metrics import f1_score 
from sklearn.utils import class_weight
from gensim.models import Word2Vec 
import wandb

use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#paramerters for filtering and creation of samples
mode='sql'
progress = 0
count = 0
restriction = [20000, 5, 6, 10]
step = 5
full_length = 200
mode_param = str(step)+"_"+str(full_length)

In [5]:
#parameters for w2v model
mincount = 5000 #minimum time a word has to appear in the corpus to be in the w2vec model
iterationen = 300  #training iterations for the w2v model
s = 200 #d
w = "withString"
w2v_name = "word2vec_" + w + str(mincount) + "-" + str(iterationen) + "-" + str(s)
w2v_model = 'w2v/'+ w2v_name + ".model"

w2v_model = Word2Vec.load(w2v_model)


In [6]:
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
# import json

# all_blocks = []
# with open('data/plain_' + mode, 'r') as infile:
  # data = json.load(infile)
# allblocks = []
# for r in data:
#   progress = progress + 1
#   for c in data[r]:
    
#     if "files" in data[r][c]:                      
#       # if len(data[r][c]["files"]) > restriction[3]:
#       #   # too many files
#       #  continue
      
#       for f in data[r][c]["files"]:
        
#         # if len(data[r][c]["files"][f]["changes"]) >= restriction[2]:
#         #   #too many changes in a single file
#         #   continue
        
#         if not "source" in data[r][c]["files"][f]:
#           #no sourcecode
#           continue
        
#         if "source" in data[r][c]["files"][f]:
#           sourcecode = data[r][c]["files"][f]["source"]                          
#           # if len(sourcecode) > restriction[0]:
#           #   #sourcecode is too long
#           #   continue
          
#           allbadparts = []
          
#           for change in data[r][c]["files"][f]["changes"]:
#                 #get the modified or removed parts from each change that happened in the commit                  
#                 badparts = change["badparts"]
#                 count = count + len(badparts)
#                 # if len(badparts) > restriction[1]:
#                 #   #too many modifications in one change
#                 #   break
                 
                
#                 for bad in badparts:
#                   #check if they can be found within the file
#                   pos = myutils.findposition(bad,sourcecode)
#                   if not -1 in pos:
#                       allbadparts.append(bad)
#                 # if (len(allbadparts) > restriction[4]):
#                 #   # too many bad positions in the file
#                 #   break
#           if(len(allbadparts) > 0):
#             # if len(allbadparts) < restriction[2]:
#               # find the positions of all modified parts
#               positions = myutils.findpositions(allbadparts,sourcecode)

#               #get the file split up in samples
#               blocks = myutils.getblocks(sourcecode, positions, step, full_length)
              
#               for b in blocks:
#                   #each is a tuple of code and label
#                   allblocks.append(b)


In [8]:
import torch
from torch.utils.data import Dataset
from torchtext.transforms import ToTensor, PadTransform
import random
class MyDataset(Dataset):
  def __init__(self, allblocks ,data_split='train', transform=None, target_transform=None):
    self.allblocks = allblocks 
    self.transform = transform
    self.target_transform = target_transform
    self.data_split=data_split
  
    keys = []
    for i in range(len(self.allblocks)):
      keys.append(i)
    random.shuffle(keys)
    cutoff = round(0.7 * len(keys)) #     70% for the training set
    cutoff2 = round(0.85 * len(keys)) #   15% for the validation set and 15% for the final test set
    self.keystrain = keys[:cutoff]
    self.keysvalid = keys[cutoff:cutoff2]
    self.keystest = keys[cutoff2:]

    default_idx = self.keystrain[0]
    block = self.allblocks[default_idx]
    code = block[0]
    self.default_token = myutils.getTokens(code)

  def __len__(self):
    if self.data_split == 'train': 
      return len(self.keystrain)
    elif self.data_split == 'valid':
      return len(self.keysvalid)
    elif self.data_split == 'test': 
      return len(self.keystest)
    else:
      print('Invalid split')

  def __getitem__(self, idx):
    word_vectors = w2v_model.wv
    if self.data_split == 'train': 
      key_idx = self.keystrain[idx]
    elif self.data_split == 'valid':
      key_idx = self.keysvalid[idx]
    elif self.data_split == 'test':
      key_idx = self.keystest[idx]
    block = self.allblocks[key_idx]
    code = block[0]
    token = myutils.getTokens(code) #get all single tokens from the snippet of code
    if(len(token) == 0): 
      print(len(token))
      token = self.default_token

    vectorlist = []
    if block[1] == 1: 
      label = 0 
    else: 
      label = 1
    for t in token: #convert all tokens into their word2vec vector representation
      if t in word_vectors.vocab and t != " ":
        word_vector = w2v_model[t]
        vectorlist.append(word_vector) 
    vectorlist = torch.tensor(vectorlist)
    if self.transform:
      vectorlist = self.transform(vectorlist)
    if self.target_transform:
      label = self.target_transform(label)
    return vectorlist, label 


In [9]:
# from torchtext.transforms import ToTensor, PadTransform

# train_dataset = MyDataset(
#     allblocks=allblocks,
#     data_split='train',
# )
# valid_dataset = MyDataset(
#     allblocks=allblocks,
#     data_split='valid'
# )
# test_dataset = MyDataset(
#     allblocks=allblocks,
#     data_split='test'
# )


In [10]:
# data_path = '/content/drive/MyDrive/data'
# isFiltered = False 
# name = '_filtered' if isFiltered else '' 
# torch.save(train_dataset, f'{data_path}/train_data_{mode}{name}.pth')
# torch.save(valid_dataset, f'{data_path}/valid_data_{mode}{name}.pth')
# torch.save(test_dataset, f'{data_path}/test_data_{mode}{name}.pth')

In [11]:
# train_dataset = torch.load(f'{data_path}/train_data_{mode}{name}.pth')
# valid_dataset = torch.load(f'{data_path}/valid_data_{mode}{name}.pth')
# test_dataset = torch.load(f'{data_path}/test_data_{mode}{name}.pth')

In [12]:
from torch.nn.utils.rnn import pad_sequence

class MyCollate:
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx
        
    
    def __call__(self, batch):
      source = []
      target = []
      for i in range(len(batch)):
        if batch[i][0].shape[0] != 0:
          source.append(batch[i][0]) 
          target.append(batch[i][1])

      for i in range(len(batch) - len(source)):
        source.append(source[-1])
        target.append(target[-1])
      
      source = pad_sequence(source, batch_first=True, padding_value = self.pad_idx) 
      return source, target 

In [13]:
from torch.utils.data import DataLoader 

def get_data_loader(dataset, batch_size, num_workers=0, shuffle=True, pin_memory=True): #increase num_workers according to CPU
    pad_idx = 0 
    loader = DataLoader(
        dataset,
        batch_size = batch_size, 
        num_workers = num_workers,
        shuffle=shuffle,
        pin_memory=pin_memory, 
        collate_fn = MyCollate(pad_idx=pad_idx)
    ) 
    return loader

In [14]:
class MyGRU(nn.Module):
  def __init__(self, hidden_size=200, num_layers=1, bidirectional=False, dropout=0.2):
    super(MyGRU, self).__init__()
    self.hidden_size = hidden_size
    self.embedding = w2v_model
    self.num_layers = num_layers
    self.bidirectional = bidirectional
    self.gru = nn.GRU(
        hidden_size,
        hidden_size,
        dropout=dropout,
        batch_first=True, 
        num_layers=self.num_layers,
        bidirectional=self.bidirectional
    )
    self.out = nn.Linear(hidden_size, 1)
    self.sigmoid = nn.Sigmoid()
  
  def forward(self, input, hidden):
    hidden = hidden.cuda()
    output, hidden = self.gru(input, hidden )
    output = self.out(output)
    output = self.sigmoid(output)
    return output, hidden
  
  def initHidden(self, batch_size=1):
    hidden = torch.zeros(2 if self.bidirectional else 1 * self.num_layers, batch_size ,self.hidden_size).to(device)
    return hidden 
    

In [27]:
class MyLSTM(nn.Module):
  def __init__(self, hidden_size=200, num_layers=1, bidirectional=False, dropout=0.2):
    super(MyLSTM, self).__init__()
    self.hidden_size = hidden_size
    self.embedding = w2v_model
    self.num_layers = num_layers
    self.bidirectional = bidirectional
    self.lstm = nn.LSTM(
        hidden_size,
        hidden_size,
        dropout=dropout,
        batch_first=True, 
        num_layers=self.num_layers,
        bidirectional=self.bidirectional
    )
    self.out = nn.Linear(hidden_size, 1)
    self.sigmoid = nn.Sigmoid()
  
  def forward(self, input, hidden):
    output, hidden = self.lstm(input, hidden)
    output = self.out(output)
    output = self.sigmoid(output)
    return output, hidden
  
  def initHidden(self, batch_size=1):
    h0 = torch.zeros(2 if self.bidirectional else 1 * self.num_layers, batch_size ,self.hidden_size).to(device)
    c0 = torch.zeros(2 if self.bidirectional else 1 * self.num_layers, batch_size ,self.hidden_size).to(device)
    return h0, c0 
    

In [28]:
import math
import time
print_every = 1000
plot_every = 50000

def timeSince(since):
  now = time.time()
  s = now - since
  m = math.floor(s/60)
  s -= m * 60
  return f'{m}m {math.floor(s)}s' 


In [29]:
def weighted_binary_cross_entropy(output, target, weights=None):
    # print(output, target)
    if weights is not None:
        assert len(weights) == 2
        loss = weights[1] * (target * torch.log(output)) + \
               weights[0] * ((1 - target) * torch.log(1 - output))
    else:
        loss = target * torch.log(output) + (1 - target) * torch.log(1 - output)

    return torch.neg(torch.mean(loss))

In [42]:

from torchmetrics import F1Score, Precision, Recall

def evaluate(model, loader, class_weights, epoch):
  print('____________________eval______________________')
  targets = torch.tensor([]).cuda() 
  preds = torch.tensor([]).cuda() 
  start = time.time()
  all_losses = []

  
  with torch.no_grad():
    print_count = 0 
    for iter, (input , labels) in enumerate(loader):

      hidden = model.initHidden(batch_size=len(labels))
      label_tensor = torch.tensor(labels).to(device)
      input = input.to(device)

      targets = torch.cat((targets, torch.tensor(label_tensor)))
      output, hidden = model(input, hidden)
      final_output = output[:, -1, :].T

      loss = weighted_binary_cross_entropy(final_output , label_tensor, weights=class_weights)
      all_losses.append(loss)
      preds = torch.cat((preds, final_output), 1)

    eval_loss = torch.mean(torch.tensor(all_losses))
    preds = torch.squeeze(preds)
    targets = targets.int()
    
  preds = torch.tensor(preds).to(device)
  targets = torch.tensor(targets).to(device)
  f1 = F1Score(num_classes=1).to(device)

  precision = Precision(num_classes=1).to(device)
  recall = Recall(num_classes=1).to(device)

  f1_score = f1(preds, targets)
  precision_score = precision(preds, targets)
  recall_score = recall(preds, targets)

  wandb.log({
    'epoch': epoch,
    'f1 score': f1_score,
    'precision score': precision_score,
    'recall score': recall_score,
    'eval loss': eval_loss
  })


In [43]:
import torch.optim as optim
def train_mini_batch(model, input_tensor, label_tensor, optimizer, class_weights):
  hidden = model.initHidden(batch_size=len(label_tensor))
  model.zero_grad()
  label_tensor = torch.tensor([label_tensor]).to(device)
  input = input_tensor.to(device)
  output, hidden = model(input, hidden)
  final_output = output[:, -1, :].T
  loss = weighted_binary_cross_entropy(final_output , label_tensor, weights=class_weights)
  loss.backward()
  optimizer.step()
  return final_output, loss.item()
  

In [44]:
def train(model, loader, optimizer, class_weights, epoch):
  print('____________________train______________________')
  all_losses = []
  for iter, (input_tensor, label_tensor) in enumerate(loader):
    preds, loss = train_mini_batch(model, input_tensor, label_tensor, optimizer, class_weights)
    all_losses.append(loss)
    if iter % 20 == 0:
      print(f'current iter: {iter}, {int(iter/len(loader)* 100)}%')
      wandb.log({'train_loop_loss': torch.mean(torch.tensor(all_losses))})

    
  wandb.log({'train_loss': torch.mean(torch.tensor(all_losses)), 'epoch': epoch}) 


In [45]:
def getModel(architecture):
  if architecture == 'GRU':
    return MyGRU().to(device)
  else:
    return MyLSTM().to(device) 

In [46]:
model = getModel('LSTM') 
datasets = ['xss', 'xsrf', 'path_disclosure', 'open_redirect', 'command_injection', 'remote_code_execution', 'sql']
architectures = ['LSTM', 'GRU']
data_path = '/content/drive/MyDrive/data'

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [47]:
import json
import numpy as np

def load_class_weights(data_path):
  class_weights = json.load(open(f'{data_path}/class_weights.json'))  
  for key in class_weights:
    class_weights[key] = np.array(class_weights[key])
  return class_weights



In [48]:
from sklearn.utils.class_weight import compute_class_weight
def run():
  conf = dict(
    learning_rate = 0.0005,
    infra = 'Google colab',
    dataset = '',
    architecure = '',
    log_interval=1000,
    epochs = 20,
    batch_size=128, 
    full_length = 200,
    step = 5,
    word_min_count = 5000,
    iterationen = 300,
    hidden_size = 200,
    with_string = 'withString',
    seed = 42,
  )

  for dataset in datasets:   
    print(f'current dataset: {dataset}')
    conf['dataset'] = dataset

    train_dataset = torch.load(f'{data_path}/train_data_{conf["dataset"]}.pth')
    train_loader = get_data_loader(train_dataset, batch_size=conf['batch_size'])

    valid_dataset = torch.load(f'{data_path}/valid_data_{conf["dataset"]}.pth')
    valid_loader = get_data_loader(valid_dataset, batch_size=conf['batch_size'])

    # test_dataset = torch.load(f'{data_path}/test_data_{conf["dataset"]}.pth')
    # test_loader = get_data_loader(test_dataset, batch_size=conf['batch_size'])
    class_weights = load_class_weights(data_path)


    for architecture in architectures:
      print(f'current architecture: {architecture}')
      conf['architecture'] = architecture
      wandb.init(
        name=f"{dataset}_{architecture}",
        project='Thesis 2',
        notes=f'Tweak baseline' ,
        entity='eddiechen372',
        tags=[architecture],
        config=conf

      )
      model = getModel(architecture)
      optimizer = optim.Adam(model.parameters(), lr=conf['learning_rate'],  weight_decay=0)
      wandb.watch(model, log='all')

      for i in range(conf['epochs']):
        wandb.log({'epoch': i})
        train(model, train_loader, optimizer, class_weights[dataset], i)
        evaluate(model, valid_loader, class_weights[dataset], i)
      


In [ ]:
run()

current dataset: xss
current architecture: LSTM


epoch,▁
train_loop_loss,▁█▇▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
epoch,0
train_loop_loss,0.69338


____________________train______________________


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


current iter: 0, 0%
current iter: 20, 6%
current iter: 40, 13%
current iter: 60, 19%
current iter: 80, 26%
current iter: 100, 33%
current iter: 120, 39%
current iter: 140, 46%
current iter: 160, 52%
current iter: 180, 59%
current iter: 200, 66%
current iter: 220, 72%
current iter: 240, 79%
current iter: 260, 86%
current iter: 280, 92%
current iter: 300, 99%
____________________eval______________________


...

____________________train______________________
current iter: 0, 0%
current iter: 20, 3%
current iter: 40, 7%
current iter: 60, 11%
current iter: 80, 15%
current iter: 100, 19%
current iter: 120, 22%
current iter: 140, 26%
current iter: 160, 30%
current iter: 180, 34%
current iter: 200, 38%
current iter: 220, 41%
current iter: 240, 45%
current iter: 260, 49%
current iter: 280, 53%
current iter: 300, 57%
current iter: 320, 60%
current iter: 340, 64%
current iter: 360, 68%
current iter: 380, 72%
current iter: 400, 76%
current iter: 420, 79%
current iter: 440, 83%
current iter: 460, 87%
current iter: 480, 91%
current iter: 500, 95%
current iter: 520, 98%
____________________eval______________________
____________________train______________________
current iter: 0, 0%
current iter: 20, 3%
current iter: 40, 7%
current iter: 60, 11%
current iter: 80, 15%
current iter: 100, 19%
current iter: 120, 22%
current iter: 140, 26%
current iter: 160, 30%
current iter: 180, 34%
current iter: 200, 38%
c

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
eval loss,█▆▅▄▄▅▃▃▃▂▂▁▁▁▁▃▁▁▁▁
f1 score,▁▂▃▄▃▃▅▅▅▇▇▇██▇▄████
precision score,▁▁▂▃▂▂▄▅▅▆▇▆█▇▇▃▇███
recall score,▁▆▆▇▇▇█▇▇█▇█████████
train_loop_loss,██▆▆▄▄▄▄▃▄▃▅▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▂▂▂▁▁▁▁▁▁
train_loss,█▆▄▄▄▄▃▂▂▂▁▁▁▁▁▂▂▁▁▁
epoch,19
eval loss,0.04358
f1 score,0.91537
precision score,0.84917


____________________train______________________
current iter: 0, 0%
current iter: 20, 3%
current iter: 40, 7%
current iter: 60, 11%
current iter: 80, 15%
current iter: 100, 19%
current iter: 120, 22%
current iter: 140, 26%
current iter: 160, 30%
current iter: 180, 34%
current iter: 200, 38%
current iter: 220, 41%
current iter: 240, 45%
current iter: 260, 49%
current iter: 280, 53%
current iter: 300, 57%
current iter: 320, 60%
current iter: 340, 64%
current iter: 360, 68%
current iter: 380, 72%
current iter: 400, 76%
current iter: 420, 79%
current iter: 440, 83%
current iter: 460, 87%
current iter: 480, 91%
current iter: 500, 95%
current iter: 520, 98%
____________________eval______________________
____________________train______________________
current iter: 0, 0%
current iter: 20, 3%
current iter: 40, 7%
current iter: 60, 11%
current iter: 80, 15%
current iter: 100, 19%
current iter: 120, 22%
current iter: 140, 26%
current iter: 160, 30%
current iter: 180, 34%
current iter: 200, 38%
c

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
eval loss,█▄▃▂▂▂▂▁▁▁▁▃▁▁▂▁▁▁▁▁
f1 score,▁▃▅▅▆▇▇▇▇██▇██▆███▇█
precision score,▁▂▄▄▅▇▇▇▇▇██▇█▆███▇█
recall score,▁▇▇████████▆██▇█████
train_loop_loss,██▅▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁
train_loss,█▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,19
eval loss,0.03451
f1 score,0.96094
precision score,0.93323


____________________train______________________
current iter: 0, 0%
current iter: 20, 1%
current iter: 40, 2%
current iter: 60, 3%
current iter: 80, 5%
current iter: 100, 6%
current iter: 120, 7%
current iter: 140, 8%
current iter: 160, 10%
current iter: 180, 11%
current iter: 200, 12%
current iter: 220, 14%
current iter: 240, 15%
current iter: 260, 16%
current iter: 280, 17%
current iter: 300, 19%
current iter: 320, 20%
current iter: 340, 21%
current iter: 360, 23%
current iter: 380, 24%
current iter: 400, 25%
current iter: 420, 26%
current iter: 440, 28%
current iter: 460, 29%
current iter: 480, 30%
current iter: 500, 32%
current iter: 520, 33%
current iter: 540, 34%
current iter: 560, 35%
current iter: 580, 37%
current iter: 600, 38%
current iter: 620, 39%
current iter: 640, 41%
current iter: 660, 42%
current iter: 680, 43%
current iter: 700, 44%
current iter: 720, 46%
current iter: 740, 47%
current iter: 760, 48%
current iter: 780, 50%
current iter: 800, 51%
current iter: 820, 52%
